<a href="https://colab.research.google.com/github/yehoon17/BERT_sandwich_order/blob/main/data_to_seq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

# data 불러오기
data = pd.Series(pd.read_table('data.csv', header=None, encoding='euc-kr', sep='\n')[0])
data[:5]

0     길이는 /length;15cm/로 할게요.
1    소스는 /sauce;스위트 칠리/로 할게요.
2         /sauce;마요네즈/로 해주세요.
3     길이는 /length;30cm/로 주세요.
4     길이는 /length;15센치/로 주세요.
Name: 0, dtype: object

In [2]:
# data를 문자로만 구성되게 만드는 함수

import re

def get_raw_text(data):
    pat = re.compile("/\w+;")
    data = pat.sub("",data)
    pat = re.compile("/")
    data = pat.sub("",data)
    pat = re.compile("[^\w\s]")
    data = pat.sub("",data)
    
    return data

In [3]:
raw_text_data = data.apply(get_raw_text)
raw_text_data[:5]

0      길이는 15cm로 할게요
1    소스는 스위트 칠리로 할게요
2         마요네즈로 해주세요
3      길이는 30cm로 주세요
4      길이는 15센치로 주세요
Name: 0, dtype: object

In [4]:
!pip install tensorflow==1.15

     |████████████████████████████████| 412.3 MB 23 kB/s 
     |████████████████████████████████| 3.8 MB 33.4 MB/s 
     |████████████████████████████████| 503 kB 38.6 MB/s 
     |████████████████████████████████| 50 kB 6.4 MB/s 
  Created wheel for gast: filename=gast-0.2.2-py3-none-any.whl size=7553 sha256=bfddc1fe97f408f2b1487836707c0485455444fcd271b73a5cd01d398b7fc0af
  Stored in directory: /root/.cache/pip/wheels/21/7f/02/420f32a803f7d0967b48dd823da3f558c5166991bfd204eef3
Successfully built gast
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.5.0
    Uninstalling tensorflow-estimator-2.5.0:
      Successfully uninstalled tensorflow-estimator-2.5.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.5.0
    Uninstalling tensorboard-2.5.0:
      Successfully uninstalled tensorboard-2.5.0
  Attempting uninstall: gast
    Found existing installation: gast 0.4.0
    Uninstalling gast-0.4.0:
      Succes

In [5]:
import tensorflow as tf
print(tf.__version__)
# 2.x 이 출력될 경우, 런타임 재시작 후 다시 확인

1.15.0


In [6]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [7]:
# 드라이브 내 test 파일 경로 설정
% cd /content/drive/MyDrive/Colab Notebooks

# 현재 디렉토리 내 파일 출력
% ls 

/content/drive/MyDrive/Colab Notebooks
 CNN.ipynb                               spam_detection.ipynb
 data_gen2                               stock.ipynb
 data_gen.ipynb                          tokenizationK.py
 data_to_seq.ipynb                       Untitled
 GRU.ipynb                               Untitled0.ipynb
 imdb_classification.ipynb              'Untitled (1)'
 __pycache__/                            vocab.korean.rawtext.list
 Python분석라이브러리활용_이예훈.ipynb   YOLO.ipynb
 seq_in.txt


In [8]:
from tokenizationK import FullTokenizer
# ETRI 버트를 받으면 tokenization.py가 들어있는데 그거 말고 꼭 제가 제공해드린 tokenizationK.py를 쓰도록 해주세요.
tokenizer = FullTokenizer(vocab_file="/content/drive/MyDrive/Colab Notebooks/vocab.korean.rawtext.list")

In [9]:
tokenized_data = raw_text_data.apply(tokenizer.tokenize)
tokenized_data[0]

['길', '이는_', '15', 'c', 'm', '로_', '할', '게', '요_']

In [10]:
# seq_in.txt로 저장하기
tokenized_data.to_csv("seq_in.txt")

In [87]:
# seq_out 생성

def get_seq_out(data, seq_in):
  seq_out = []
  for token in seq_in:
    pat = re.compile("/\w+;\w*"+token+"\w*/")
    m = pat.search(data)
    if m:
      slot_pat = re.compile("/\w+;")
      slot_match = slot_pat.match(m.group())
      seq_out.append(slot_match.group()[1:-1])
    else:
      seq_out.append(0)
  
  return seq_out

In [90]:
seq_out = []
for sentence, seq_in in zip(data, tokenized_data):
  seq_out.append(get_seq_out(sentence, seq_in))

seq_out[:5]

[[0, 0, 'length', 'length', 'length', 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 ['sauce', 'sauce', 'sauce', 'sauce', 0, 0, 0],
 [0, 0, 'length', 'length', 'length', 0, 0, 0],
 [0, 0, 'length', 'length', 'length', 0, 0, 0]]

In [93]:
data[:5]

0     길이는 /length;15cm/로 할게요.
1    소스는 /sauce;스위트 칠리/로 할게요.
2         /sauce;마요네즈/로 해주세요.
3     길이는 /length;30cm/로 주세요.
4     길이는 /length;15센치/로 주세요.
Name: 0, dtype: object

In [94]:
tokenized_data[:5]

0        [길, 이는_, 15, c, m, 로_, 할, 게, 요_]
1    [소, 스는_, 스, 위, 트_, 칠, 리로_, 할, 게, 요_]
2               [마, 요, 네, 즈, 로_, 해주, 세요_]
3          [길, 이는_, 30, c, m, 로_, 주, 세요_]
4          [길, 이는_, 15, 센, 치, 로_, 주, 세요_]
Name: 0, dtype: object